[View the runnable example on GitHub](https://github.com/intel-analytics/BigDL/tree/main/python/nano/tutorial/notebook/inference/tensorflow/accelerate_tensorflow_inference_openvino.ipynb)

# Accelerate TensorFlow Inference using OpenVINO

You can use ``InferenceOptimizer.trace(..., accelerator='openvino')`` API to enable the OpenVINO acceleration for TensorFlow inference. It only takes a few lines.

To apply OpenVINO acceleration, the following dependencies need to be installed first:

In [ ]:
!pip install --pre --upgrade bigdl-nano[tensorflow,inference] # install the nightly-built version
!source bigdl-nano-init # set environment variables

> 📝 **Note**
>
>  We recommend to run the commands above, especially `source bigdl-nano-init` before jupyter kernel is started, or some of the optimizations may not take effect.

Let's take an [EfficientNetB0 model](https://www.tensorflow.org/api_docs/python/tf/keras/applications/efficientnet/EfficientNetB0) pretrained on ImageNet dataset as an example. First, we load the model:

In [ ]:
from tensorflow.keras.applications import EfficientNetB0

model = EfficientNetB0(weights='imagenet')

In [2]:
model.summary()

Model: "efficientnetb0"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 rescaling (Rescaling)          (None, 224, 224, 3)  0           ['input_1[0][0]']                
                                                                                                  
 normalization (Normalization)  (None, 224, 224, 3)  7           ['rescaling[0][0]']              
                                                                                                  
 tf.math.truediv (TFOpLambda)   (None, 224, 224, 3)  0           ['normalization[0][0

To enable OpenVINO acceleration for your TensorFlow inference, **the only change you need to made is to import BigDL-Nano** `InferenceOptimizer`**, and trace your TensorFlow model to convert it into an OpenVINO accelerated module for inference**:

In [ ]:
import tensorflow as tf
from bigdl.nano.tf.keras import InferenceOptimizer

ov_model = InferenceOptimizer.trace(model,
                                    accelerator="openvino",
                                    input_spec=tf.TensorSpec(shape=(None, 224, 224, 3))
                                    )

> 📝 **Note**
>
> Different from the case when using ONNXRuntime as accelerator, `input_spec` is an **optional** argument for the case when `accelerator="openvino"`, as OpenVINO can infer the input shape for a model created by Keras Functional or Sequential API.
>
> However, when users create a custom model (e.g. inherated from `tf.keras.Model`), the `input_spec` is still needed to for OpenVINO accelerator to know the **shape** of the model input.
>
>  Please refer to [API documentation](https://bigdl.readthedocs.io/en/latest/doc/PythonAPI/Nano/tensorflow.html#bigdl.nano.tf.keras.InferenceOptimizer.trace) for more information on `InferenceOptimizer.trace`.

You could then do the normal inference steps with the model optimized by OpenVINO:

In [ ]:
x = tf.random.normal(shape=(2, 224, 224, 3))
# use the optimized model here
y_hat = ov_model(x)
predictions = tf.argmax(y_hat, axis=1)
print(predictions)

> 📚 **Related Readings**
> 
> - [How to install BigDL-Nano](https://bigdl.readthedocs.io/en/latest/doc/Nano/Overview/nano.html#install)
> - [How to install BigDL-Nano in Google Colab](https://bigdl.readthedocs.io/en/latest/doc/Nano/Howto/install_in_colab.html)